## 각 제품(앱)에 들어가서 리뷰를 스크롤링 하기 

In [8]:
from selenium import webdriver
browser = webdriver.Chrome(executable_path='../../../../chromedriver.exe') 
browser.get('https://play.google.com/store/search?q=%ED%97%AC%EC%8A%A4%EC%BC%80%EC%96%B4%20%EC%95%B1&c=apps&hl=ko-KR')

In [9]:
browser.implicitly_wait(10)

In [10]:
import pandas as pd 
import pymongo as mg
import time

In [ ]:
browsernt = mg.MongoClient(host='mongodb://localhost:27017')
database = client['study_data_analytics']
collection = database['googlestore_heathcare']

# 서비스 앱에 대한 리스트 가져오기 
click_path = 'section > div > div > div > div' 
company_list = browser.find_elements_by_css_selector(click_path)

len(company_list)

for company in company_list:
    company.click()
    time.sleep(2) #term을 주기 
    
    # 서비스 앱에 대한 정보 가져오기 
    try : 
        service_name = browser.find_element_by_css_selector('div > h1 > span').text
    except: 
        service_name = str()
    try : 
        company_name = browser.find_element_by_css_selector('div.Il7kR > div > div > div > a > span').text
    except: 
        company_name = str()
    try : 
        app_information = browser.find_element_by_css_selector('div:nth-child(1) > c-wiz:nth-child(2) > div > section > div').text
    except : 
        app_information = str()
        
    # [service_name, company_name, app_information]
    
    # 댓글 팝업 띄우기(해당 리뷰 없을 시 대비한 방어 코드 작성 필요)
    review_flag = True
    try : 
        browser.find_element_by_xpath('//*/div/div/button/*[contains(text(), "리뷰")]').click() 
    except : 
        review_flag = False
        
    # 회사 서비스에 대한 전체 리뷰 가져오기 
    serviceapp_colums_name = ['service_name', 'content', 'date', 'stars']
    reviews_list = list()
    if review_flag : # 리뷰가 있으면 
        reviews_bundle = browser.find_elements_by_css_selector('div.fysCi > div > div:nth-child(2) > div')
        len(reviews_bundle)

        for review_bundle in reviews_bundle : 
            content = review_bundle.find_element_by_css_selector('div.h3YV2d').text
            date = review_bundle.find_element_by_css_selector('header > div.Jx4nYe > span').text
            stars = review_bundle.find_element_by_css_selector('header > div.Jx4nYe > div').get_attribute('aria-label')
            review_list = [service_name, content, date, stars]
            reviews_list.append(review_list) # 리뷰에 대한 모든 것(특정 회사 서비스에 대한)
        
        browser.back() #팝업 닫기
        
        # 저장을 위한 작업 
        df_reviews = pd.DataFrame(data=reviews_list, columns=serviceapp_colums_name)
        data_dict = df_reviews.to_dict(orient='records')
        collection.insert_many(data_dict)
        
    print('service_name : {}, review count : {}'.format(service_name, len(reviews_list)))
    browser.back() # 리스트로 돌아가기 
    time.sleep(3)
print('Done')

service_name : Samsung Health(삼성 헬스), review count : 40
service_name : NH헬스케어, review count : 40
service_name : 케어헬스, review count : 17
service_name : 채움건강, review count : 40
service_name : 헬스케어 건강관리서비스, review count : 1
service_name : 기업용 디지털 헬스케어 삼성화재 애니핏 프로(PRO), review count : 7
service_name : SK VIEW 헬스케어, review count : 2
service_name : 애니핏 플러스 나에게 핏한 건강관리, review count : 40
service_name : 삼성생명 S-헬스케어, review count : 25
service_name : 오케어, review count : 40
service_name : 어떠케어 - 내 몸이 궁금할 때, review count : 40
service_name : 캐시워크 - 적립형 만보기 첫화면, review count : 40
service_name : 오늘건강 - 어르신 건강관리 앱, review count : 40
service_name : 닥터다이어리 – 식단, 체중, 혈당, 혈압관리 필수 앱, review count : 40
service_name : Health4U, review count : 40
service_name : 케어네이션 - 간병인, 병원 동행인, 요양보호사 찾기, review count : 40
service_name : 더헬스 (THE Health), review count : 40


In [ ]:
browser.quit()